# Evaluating model and interpret the classification results

In [1]:
import torchray
import torch
import attr
from pathlib import Path
from torchvision import datasets, transforms
from torchray.benchmark import get_example_data, plot_example
from torchvision import models
import torchvision
import matplotlib.pyplot as plt
import numpy as np
from sklearn import metrics as sk_metrics
from torchray.attribution.guided_backprop import GuidedBackpropReLU
from torchray.attribution.common import Probe, get_module
from torchray.attribution.grad_cam import grad_cam
from torchray.attribution.grad_cam import gradient_to_grad_cam_saliency
from PIL import Image
import cv2

In [2]:
labels_bin = {"abnormal": ["A", "a", "J", "S", "V", "E", "F", "P", "/", "f", "Q"], "normal": ["N", "L", "R", "e", "j"]}

In [3]:
labels_multi = {"SVEB": ["A", "a", "J", "S"], "VEB": ["V", "E"], "F": ["F"], "Q": ["P", "/", "f", "Q"], "N": ["N", "L", "R", "e", "j"]}

In [4]:
MULTI = False
if MULTI:
    for label in labels_multi:
        Path(f"../GB_Grad_CAM_maps/MULTI_label_{beat}_beat/{label}").mkdir(parents=True)
else:
    for label in labels_bin:
        Path(f"../GB_Grad_CAM_maps/label_{beat}_beat/{label}").mkdir(parents=True)

NameError: name 'beat' is not defined

In [5]:
@attr.s(auto_attribs=True)
class DataPreparation:
    data_dir: Path
    device: str = attr.ib(default=torch.device("cuda:0" if torch.cuda.is_available() else "cpu"), init=False)

    @staticmethod
    def data_transformations():
        data_transforms = {
            'test': transforms.Compose([
                transforms.CenterCrop((200, 1500)),
                transforms.Resize((224, 224)),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ])
        }
        return data_transforms
    
    def create_dataloaders(self, batch_size, shuffle, num_workers):
        data_transforms = self.data_transformations()
        
        image_datasets = {
            'test': datasets.ImageFolder(self.data_dir, data_transforms['test'])
        }
        dataloaders = {
            'test': torch.utils.data.DataLoader(image_datasets['test'], batch_size=batch_size, shuffle=shuffle, num_workers=num_workers)
        }
        dataset_sizes = {
            'test': len(image_datasets['test'])
        }
        return dataloaders, dataset_sizes


In [6]:
class GuidedBackpropReLUModel:
    def __init__(self, model):
        self.model = model
#         self.model.eval()

        def recursive_relu_apply(module_top):
            for idx, module in module_top._modules.items():
                recursive_relu_apply(module)
                if module.__class__.__name__ == 'ReLU':
                    module_top._modules[idx] = GuidedBackpropReLU.apply

        # replace ReLU with GuidedBackpropReLU
        recursive_relu_apply(self.model)

    def forward(self, input_img):
        return self.model(input_img)

    def __call__(self, input_img, target_category=None):

        input_img = input_img.requires_grad_(True)

        output = self.forward(input_img)

        if target_category == None:
            target_category = np.argmax(output.cpu().data.numpy())

        one_hot = np.zeros((1, output.size()[-1]), dtype=np.float32)
        one_hot[0][target_category] = 1
        one_hot = torch.from_numpy(one_hot).requires_grad_(True)

        one_hot = torch.sum(one_hot * output)
        one_hot.backward(retain_graph=True)

        output = input_img.grad.cpu().data.numpy()
        output = output[0, :, :, :]

        return output

In [7]:
variants = ["final", "mid", "initial",  "multiclass"]
models = ["resnet50_d_02_t_20_10_final_beat", "resnet50_d_02_t_22_52_mid_beat", "resnet50_d_02_t_20_28_initial_beat", "resnet50_d_05_t_00_40"]

In [8]:
index = 0 

In [9]:
data_prep = DataPreparation(f'../data/figures_{variants[index]}/test')
data, size = data_prep.create_dataloaders(16, False, 4)

In [10]:
model_path = Path().cwd().parents[0] / f"models/{models[index]}.pth"
model = torch.load(model_path)
model.eval();

-------------------------------------------------------------------------------------------------

### Guided Back Propagation

In [11]:
def deprocess_image_gb(img):
    """ see https://github.com/jacobgil/keras-grad-cam/blob/master/grad-cam.py#L65 """
    img = img - np.mean(img)
    img = img / (np.std(img) + 1e-5)
    img = img * 0.1
    img = img + 0.5
    img = np.clip(img, 0, 1)
    return np.uint8(img*255)

In [12]:
gb_model = GuidedBackpropReLUModel(model=model.cpu())
i = 0
for inputs, labels in data['test']:
    inputs = inputs.to(0)
    labels = labels.to(0)
    x = inputs
#     break
    x.requires_grad_();
    saliency_layer = get_module(model, model.layer4)
    probe = Probe(saliency_layer, target='output')
    y = model(x)
    score_max_index = y.argmax(dim=1)
    z = y[:, score_max_index]
    z.backward(torch.ones_like(z))
    saliency = gradient_to_grad_cam_saliency(probe.data[0])
    
    for index in range(len(saliency)): 
        plt.figure()
        heatmap = np.float32(saliency[index, 0].cpu().detach())
        img = np.array(deprocess(x[index].cpu().detach()))

        heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
        heatmap = np.uint8(255 * heatmap)
        cam_mask = cv2.merge([heatmap, heatmap, heatmap])
        
        gb = gb_model(x[index].unsqueeze(0).cpu(), target_category=labels[index].cpu())
        gb = gb.transpose((1, 2, 0))
        cam_gb = deprocess_image_gb(cam_mask*gb)


        img1 = plt.imshow(cam_gb)
        
        plt.axis('off')
        if labels[index] == 0:
            label = "abnormal"
        else:
            label = "normal"
        ### CHANGE NAME!!!!!!!
        plt.savefig(f"../GB_Grad_CAM_maps/label_{beat}_beat/{label}/{i}_{index}.png")
        plt.close()
        
    i += 1

RuntimeError: Input type (torch.cuda.FloatTensor) and weight type (torch.FloatTensor) should be the same

------------------------------------------------

In [34]:
gb = GuidedBackpropReLU(model)

In [ ]:
cam_mask = cv2.merge([heatmap, heatmap, heatmap])
gb_model = GuidedBackpropReLUModel(model=model.cpu())

In [ ]:
index = 15

In [ ]:
gb = gb_model(x[index].unsqueeze(0).cpu(), target_category=labels[index].cpu())
gb = gb.transpose((1, 2, 0))

In [ ]:
cam_gb = deprocess_image_gb(cam_mask*gb)

In [ ]:
plt.imshow(cam_gb)
plt.show()